In [ ]:
import json
import re
import emoji
from joblib import Parallel, delayed
from tqdm import tqdm

# =========================================================
# ⚙️ 参数
# =========================================================
INPUT_FILE = "recipe_corpus_full.json"
OUTPUT_JSON_FILE = "recipes_cleaned_dedup.json"
MAX_TEXT_LEN = 1500   # 🔥 text 最终最大长度

# =========================================================
# 🧩 emoji 统一处理
# =========================================================
emoji_pattern = re.compile(
    "[" "\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF" "\U0001F680-\U0001F6FF"
    "\U0001F700-\U0001F77F" "\U0001F780-\U0001F7FF" "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF" "\U0001FA00-\U0001FA6F" "\U0001FA70-\U0001FAFF"
    "\U00002700-\U000027BF" "\U0001F1E0-\U0001F1FF" "]+",
    flags=re.UNICODE
)

def replace_emojis(s: str) -> str:
    if not isinstance(s, str):
        return s
    prev = {"last": None}
    def repl(match):
        em = match.group(0)[0]
        name = emoji.demojize(em, language="zh")
        name = re.sub(r'^:+|:+$', '', name)
        name = re.sub(r'[_]+', '', name).strip()
        if not name or name == em:
            name = emoji.demojize(em)
            name = re.sub(r'^:+|:+$', '', name)
            name = re.sub(r'[_]+', '', name).strip()
        if not name:
            return ''
        if name == prev["last"]:
            return ''
        prev["last"] = name
        return name
    out = re.sub(emoji_pattern, repl, s)
    out = re.sub(r'\s+', ' ', out).strip()
    return out

# =========================================================
# 🧹 文本清洗
# =========================================================
def clean_text(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = s.replace("\\n", "\n")
    s = re.sub(r'[\b\r\t]', '', s)
    s = re.sub(r'\\"+', '"', s)
    s = re.sub(r'""+', '', s)
    s = re.sub(r';{2,}', ';', s)
    s = replace_emojis(s)
    s = re.sub(r'^\s*图片\s*图片?\s*$', '', s, flags=re.MULTILINE)
    s = re.sub(r'[^\u4e00-\u9fa5A-Za-z0-9，。、“”‘’！；：：《》〈〉·,.!?()（）\s-]', '', s)
    s = re.sub(r'[ \t\f\v]+', ' ', s)
    s = re.sub(r'\s*([，。！？：；,.!?()（）])\s*', r'\1', s)
    s = s.strip()
    return s

# =========================================================
# 🧹 清理列表元素
# =========================================================
def clean_list_item(x: str) -> str:
    if not isinstance(x, str):
        return x
    x = clean_text(x)
    if "成品" in x.replace(" ", "") or "看图文中的做法" in x.replace(" ", ""):
        return ""
    return x

# =========================================================
# 🧹 清理 name/dish
# =========================================================
def clean_name(s: str) -> str:
    s = clean_text(s)
    s = re.sub(r'[\(\（][^\w\u4e00-\u9fa5]*[\)\）]', '', s)
    return s.strip()

# =========================================================
# 🔥 构建 text（顺序 & 截断 1500 字）
# =========================================================
def process_line(line, count):
    line = line.strip()
    if not line:
        return None

    try:
        obj = json.loads(line)
    except:
        return None

    obj.pop("author", None)

    # 清洗所有字段
    for k, v in list(obj.items()):
        if isinstance(v, str):
            obj[k] = clean_text(v)
        elif isinstance(v, list):
            cleaned_list = [clean_list_item(x) for x in v]
            obj[k] = [x for x in cleaned_list if x]

    # dish
    dish = (obj.get("dish") or "").strip()
    if not dish or dish.lower() == "unknown":
        dish = obj.get("name", "").strip()
    dish = clean_name(dish)

    parts = []
    if dish:
        parts.append(f"dish: {dish}")

    ingredients = obj.get("recipeIngredient", [])
    if ingredients:
        parts.append("recipeIngredient: " + " ".join(ingredients))

    # recipeInstructions 放最后之一
    recipe_instructions = obj.get("recipeInstructions", [])
    if recipe_instructions:
        steps_lines = [f"{i+1}.{x}" for i, x in enumerate(recipe_instructions)]
        instructions_block = "recipeInstructions: " + " ".join(steps_lines)
    else:
        instructions_block = ""

    # keywords 放 instructions 前
    keywords = obj.get("keywords", [])
    if keywords:
        parts.append("keywords: " + "，".join(keywords))

    # description 放 instructions 前（倒数第二）
    description = obj.get("description", "")
    description_block = f"description: {description}" if description else ""

    # 合成 text：保持你要求的顺序
    text_parts = []
    text_parts.extend(parts)                # dish, ingredients, keywords
    if description_block:
        text_parts.append(description_block)  # 倒数第二
    if instructions_block:
        text_parts.append(instructions_block) # 最后

    text = " ".join(text_parts)

    # 去换行与多空白
    text = text.replace("\\n", " ").replace("\n", " ")
    text = re.sub(r'\s+', ' ', text).strip()

    # 🔥 TRUNCATE to 1500 characters
    if len(text) > MAX_TEXT_LEN:
        text = text[:MAX_TEXT_LEN].rstrip()

    return {
        "id": count + 1,
        "name": obj.get("name", "").strip(),
        "text": text
    }


def jaccard_similarity(s1, s2):
    set1, set2 = set(s1), set(s2)
    if not set1 or not set2:
        return 0
    return len(set1 & set2) / len(set1 | set2)

# =========================================================
# 🔥 主流程
# =========================================================
with open(INPUT_FILE, "r", encoding="utf-8") as fin:
    lines = fin.readlines()

processed = Parallel(n_jobs=-1, backend="loky")(
    delayed(process_line)(line, i) for i, line in enumerate(tqdm(lines, desc="Cleaning lines", ncols=90))
)

output_list = [x for x in processed if x]

output_list_sorted = sorted(output_list, key=lambda x: x['name'])

deduped_list = []
prev_item = None
for item in output_list_sorted:
    if prev_item is None:
        deduped_list.append(item)
        prev_item = item
    else:
        if prev_item['name'] == item['name']:
            sim = jaccard_similarity(prev_item['text'], item['text'])
            if sim < 0.8:
                deduped_list.append(item)
                prev_item = item
        else:
            deduped_list.append(item)
            prev_item = item

with open(OUTPUT_JSON_FILE, "w", encoding="utf-8") as fout:
    json.dump(deduped_list, fout, ensure_ascii=False, indent=2)

print(f"🎉 完成！共处理 {len(deduped_list)} 条记录。输出文件：{OUTPUT_JSON_FILE}")


Cleaning lines: 100%|████████████████████████| 1550151/1550151 [01:28<00:00, 17528.84it/s]


🎉 完成！共处理 1544608 条记录。输出文件：recipes_cleaned_dedup.json


In [11]:
import json
import re
from joblib import Parallel, delayed
from tqdm import tqdm

INPUT_FILE = "recipes_cleaned_dedup.json"
OUTPUT_FILE = "recipes_cleaned111_fixed.json"

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# -----------------------------
# 修复 name
# -----------------------------
def fix_name(item):
    name = item.get("name", "")
    if not name:
        return item
    # 删除所有空括号 ()、（）、[ ]、【 】，不限位置
    name = re.sub(r'\(\s*\)', '', name)
    name = re.sub(r'（\s*）', '', name)
    name = re.sub(r'\[\s*\]', '', name)
    name = re.sub(r'【\s*】', '', name)
    # 去掉开头结尾引号
    name = re.sub(r'^[\'"“”‘’]+', '', name)
    name = re.sub(r'[\'"“”‘’]+$', '', name)
    item['name'] = name.strip()
    return item

# -----------------------------
# 清理 text
# -----------------------------
def fix_text(item):
    text = item.get("text", "")
    if not text:
        return item
    # 删除所有空括号
    text = re.sub(r'\(\s*\)', '', text)
    text = re.sub(r'（\s*）', '', text)
    text = re.sub(r'\[\s*\]', '', text)
    text = re.sub(r'【\s*】', '', text)
    # 合并多空格
    text = re.sub(r'\s+', ' ', text).strip()
    item['text'] = text
    return item

# -----------------------------
# 组合处理
# -----------------------------
def process(item):
    item = fix_name(item)
    item = fix_text(item)
    return item

# -----------------------------
# 并行处理 + 进度条
# -----------------------------
results = Parallel(n_jobs=-1, backend="loky")(
    delayed(process)(item) for item in tqdm(data, desc="Processing", ncols=80)
)

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n🎉 完成！写入：{OUTPUT_FILE}（共 {len(results)} 条记录）")


Processing: 100%|███████████████████| 1544608/1544608 [02:56<00:00, 8759.30it/s]



🎉 完成！写入：recipes_cleaned111_fixed.json（共 1544608 条记录）


In [ ]:
# 查看数据条目
import json

INPUT_FILE = "recipes_cleaned111_fixed.json"
TOP_K = 100

# 读取 JSON 文件
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# 计算 text 长度
items_with_len = []
for item in data:
    text = item.get("text", "")
    length = len(text)
    items_with_len.append((length, item))

# 按长度排序
items_with_len.sort(reverse=True, key=lambda x: x[0])

# 取前 100 个
top_items = items_with_len[:TOP_K]

# 输出（你可以改成保存到文件）
for i, (length, item) in enumerate(top_items, 1):
    print(f"{i}. id={item['id']}, name={item['name']}, text_len={length}")


1. id=209584, name=(珍妮曲奇网红曲奇)咖啡小花配方烘焙视频饼干篇2, text_len=1500
2. id=1080742, name=(视频)正宗香港曲奇蛋挞 不塌馅的蛋挞, text_len=1500
3. id=984950, name=(超详细)新手100成功手工馒头紫薯味红枣味红糖味黑芝麻燕麦味 中筋面粉高筋面粉, text_len=1500
4. id=1022133, name=---直播---养天然酵种-后续喂养储存, text_len=1500
5. id=362299, name=-大虾盖饭-（小龙虾做法的汤汁版同理可通往做小龙虾）减肥人群解馋！不加主食就是一道超级下酒菜-啤酒虾-, text_len=1500
6. id=503724, name=02自制熟猫饭鸡胸猪瘦肉秋刀鱼三文鱼, text_len=1500
7. id=1039051, name=0基础0失败的巴斯克烤芝士蛋糕（附赠奶油芝士保存法）, text_len=1500
8. id=104371, name=0基础也可轻松做出的情人节专属美食, text_len=1500
9. id=822869, name=0基础保姆级不藏私系列(三)不会裂的戚风蛋糕卷(附带网红抱抱卷做法), text_len=1500
10. id=1193665, name=0基础保姆级不藏私系列(二)葱油香酥春饼(附葱油的熬制方法), text_len=1500
11. id=657645, name=0基础保姆级不藏私系列(四)手工馒头篇, text_len=1500
12. id=955188, name=0失败不用卷帘徒手卷制紫菜包饭大法, text_len=1500
13. id=739412, name=0失败冰糖川贝柠檬膏（止咳化痰）, text_len=1500
14. id=765984, name=0失败戚风蛋糕，8寸，适合懒人与小白，一次就能成功, text_len=1500
15. id=1228040, name=0失败的流心蛋包饭, text_len=1500
16. id=1374614, name=0失败酸奶小溶豆，适合6个月以上的宝宝吃（此配方只适合对蛋白不过敏的宝宝）, text_len=1500
17. id=145006, name=0色素中街

In [15]:
import json

input_file = "stage2_output.json"

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# 统计不同的食物数量（根据 id 或 name）
unique_ids = set(item["id"] for item in data)
unique_names = set(item["name"] for item in data)

print(f"总样本数（包括裁剪块）: {len(data)}")
print(f"不同食物数（按 id 统计）: {len(unique_ids)}")
print(f"不同食物数（按 name 统计）: {len(unique_names)}")


总样本数（包括裁剪块）: 1539473
不同食物数（按 id 统计）: 1539473
不同食物数（按 name 统计）: 1059441


In [ ]:
# 在name相同情况下对text进行去重
import json
import re
from joblib import Parallel, delayed
from itertools import groupby
from tqdm import tqdm
import multiprocessing
import os

import Levenshtein
from rapidfuzz.distance import LCSseq

# ============================
# 相似度函数
# ============================
def edit_similarity(a, b):
    max_len = max(len(a), len(b))
    if max_len == 0:
        return 0
    return 1 - Levenshtein.distance(a, b) / max_len

split_chars = re.compile(r'[^\w]+', re.UNICODE)
def word_set(s):
    return set([w for w in split_chars.split(s) if w])

def jaccard(A_set, B_set):
    if not A_set and not B_set:
        return 0
    return len(A_set & B_set) / len(A_set | B_set)

def lcs_ratio(a, b):
    n = max(len(a), len(b))
    if n == 0:
        return 0
    return LCSseq.normalized_similarity(a, b)

def full_similarity(a, b, setA, setB):
    return (
        0.4 * edit_similarity(a, b) +
        0.3 * jaccard(setA, setB) +
        0.3 * lcs_ratio(a, b)
    )

# ============================
# 核心文本抽取
# ============================
core_re_dish = re.compile(r"dish[:：](.*?) recipeIngredient", re.S)
core_re_instr = re.compile(r"recipeInstructions[:：](.*?)(keywords|$)", re.S)
core_re_kw = re.compile(r"keywords[:：](.*)$", re.S)

def extract_core_text(item):
    t = item.get("text", "")
    d = core_re_dish.search(t)
    i = core_re_instr.search(t)
    k = core_re_kw.search(t)
    res = []
    if d: res.append(d.group(1).strip())
    if i: res.append(i.group(1).strip())
    if k: res.append(k.group(1).strip())
    return " ".join(res)

# ============================
# 去重函数
# ============================
def deduplicate_same_name_group(group, threshold=0.7):
    cached = [{"item": item, "core": extract_core_text(item), "set": word_set(extract_core_text(item))} for item in group]
    kept = []
    for obj in cached:
        a = obj["core"]
        aset = obj["set"]
        is_dup = False
        for k in kept:
            score = full_similarity(a, k["core"], aset, k["set"])
            if score >= threshold:
                is_dup = True
                break
        if not is_dup:
            kept.append(obj)
    return [obj["item"] for obj in kept]

# ============================
# 主流程
# ============================
input_file = "recipes_cleaned111_fixed.json"
output_file = "stage1_output_line.json"
save_every = 1000  # 每处理 1000 个 group 就写一次

# 1. 载入数据
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. 排序并分组
data.sort(key=lambda x: x['name'])
groups = [list(g) for _, g in groupby(data, key=lambda x: x["name"])]

n_jobs = max(1, multiprocessing.cpu_count() - 1)
final_data = []

# 3. 按 batch line-by-line 增量处理
for i in tqdm(range(0, len(groups), save_every), desc="去重批次", ncols=90):
    batch = groups[i:i+save_every]
    results = Parallel(n_jobs=n_jobs)(
        delayed(deduplicate_same_name_group)(group) for group in batch
    )
    # flatten
    batch_flat = [item for sub in results for item in sub]
    final_data.extend(batch_flat)

    # 增量保存到新文件
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(final_data, f, ensure_ascii=False, indent=2)

print(f"\n🎉 完成！总组数 {len(groups)}，输出文件：{output_file}")


去重批次: 100%|█████████████████████████████████████| 1060/1060 [2:55:00<00:00,  9.91s/it]


🎉 完成！总组数 1059441，输出文件：stage1_output_line.json


关键词相似度去重完成，总条目： 9990


In [14]:
import json
import re
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

# ---------------- 基础工具函数 ----------------
def lev_distance(s1, s2):
    if len(s1) < len(s2):
        return lev_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    prev = list(range(len(s2) + 1))
    for i, c1 in enumerate(s1):
        curr = [i + 1]
        for j, c2 in enumerate(s2):
            ins = prev[j + 1] + 1
            dele = curr[j] + 1
            sub = prev[j] + (c1 != c2)
            curr.append(min(ins, dele, sub))
        prev = curr
    return prev[-1]

def lcs_length(a, b):
    n, m = len(a), len(b)
    dp = [[0]*(m+1) for _ in range(n+1)]
    for i in range(n):
        for j in range(m):
            dp[i+1][j+1] = dp[i][j]+1 if a[i]==b[j] else max(dp[i][j+1], dp[i+1][j])
    return dp[n][m]

def similarity(a: str, b: str) -> float:
    max_len = max(len(a), len(b))
    if max_len == 0:
        return 1.0
    ed = lev_distance(a, b)
    edit_sim = 1 - ed / max_len
    set_a, set_b = set(a), set(b)
    inter = len(set_a & set_b)
    union = len(set_a | set_b)
    jaccard = inter / union if union else 0
    lcs = lcs_length(a, b)
    lcs_ratio = lcs / max_len
    return 0.4*edit_sim + 0.3*jaccard + 0.3*lcs_ratio

def deduplicate_keywords(keywords, threshold=0.75):
    kept = []
    for kw in keywords:
        if not kept:
            kept.append(kw)
            continue
        if any(similarity(kw, k) >= threshold for k in kept):
            continue
        kept.append(kw)
    return kept

def split_keywords(raw: str):
    """拆分 keywords 字符串并过滤空值和 description 内容"""
    tokens = [k.strip() for k in re.split(r"[ ,，;；]+", raw) if k.strip()]
    tokens = [t for t in tokens if not t.lower().startswith("description")]
    return tokens

def extract_keywords_from_text(text: str):
    """从 text 中精准提取 keywords 部分"""
    match = re.search(r"keywords[:：](.*?)(?=description|recipeInstructions|$)", text, flags=re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1)
    return ""

def remove_keywords_from_text(text: str):
    """
    删除 text 中的 keywords 段，并去掉换行符和末尾多余的 'description:'
    """
    # 删除 keywords 段
    text = re.sub(r"keywords[:：].*?(?=description|recipeInstructions|$)", "", text, flags=re.IGNORECASE | re.DOTALL)
    # 替换换行符
    text = text.replace("\n", " ").strip()
    # 删除末尾多余的 description:
    text = re.sub(r"description[:：]\s*$", "", text, flags=re.IGNORECASE)
    return text


# ---------------- 处理单条记录 ----------------
def process_item(item):
    # 从 text 提取 keywords
    raw_kw = extract_keywords_from_text(item.get("text", ""))
    keywords = split_keywords(raw_kw)
    item["keywords"] = deduplicate_keywords(keywords)

    # 更新 text，删除原 keywords 段并去掉换行符
    item["text"] = remove_keywords_from_text(item.get("text", ""))

    return item

# ---------------- 主流程 ----------------
input_file = "stage1_output_line.json"
output_file = "stage2_output.json"
batch_size = 1000

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

n = len(data)
n_jobs = max(1, multiprocessing.cpu_count() - 1)
final_data = []

print(f"开始处理 {n} 条记录（并行: {n_jobs} 核心，每 {batch_size} 条增量保存）...")

for start in tqdm(range(0, n, batch_size), desc="处理批次", ncols=90):
    batch = data[start:start + batch_size]
    processed_batch = Parallel(n_jobs=n_jobs)(
        delayed(process_item)(item) for item in batch
    )
    final_data.extend(processed_batch)

    # 增量保存
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(final_data, f, ensure_ascii=False, indent=2)

print(f"完成！共处理 {len(final_data)} 条记录，已保存到 {output_file}")


开始处理 1539473 条记录（并行: 23 核心，每 1000 条增量保存）...


处理批次:  94%|█████████████████████████████▉  | 1442/1540 [6:18:52<89:18:17, 3280.59s/it]c:\Users\14480\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
处理批次: 100%|████████████████████████████████████| 1540/1540 [10:27:30<00:00, 24.45s/it]

完成！共处理 1539473 条记录，已保存到 stage2_output.json
